In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
from typing import Literal, List
import matplotlib.pyplot as plt

import IPython
import numpy as np
import pandas as pd
from IPython.display import display

notebook_path = Path(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"])
project_dir = notebook_path.parent.parent
sys.path.append(str(project_dir))


import src.utils.custom_log as custom_log
from src.utils.Csv import Csv
from src.visualization.validity_chain.ValChainStyledPlotCorridor import \
    ValChainStyledPlotCorridor

os.chdir(project_dir)
LOG = custom_log.init_logger(__name__)
custom_log.set_level(custom_log.LEVELS.INFO)
LOG.info("Start in %s", Path().cwd())

B_PATH: Path = Path("/mnt") / "q" / "Val_Chain_Sims" / "AB_Testing"
if B_PATH.is_dir():
    LOG.info("B_PATH: %s", B_PATH)
else:
    LOG.critical("B_PATH: %s does not exits", B_PATH)
P_PATH: Path = B_PATH / "corridors" / notebook_path.stem
P_PATH.mkdir(exist_ok=True, parents=True)
LOG.info("P_PATH: %s", P_PATH)

2024-06-03 13:11:42,492 MainProcess - MainThread INFO Start in /root/py_projects/aihiii || 2978487848.py
2024-06-03 13:11:42,495 MainProcess - MainThread INFO B_PATH: /mnt/q/Val_Chain_Sims/AB_Testing || 2978487848.py
2024-06-03 13:11:42,500 MainProcess - MainThread INFO P_PATH: /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors || 2978487848.py


# Get Data

In [2]:
CASES: List[str] = [
    "000_Base_Model",
    "100_Guided_BIW",
    "200_PAB_Simplified",
    "300_Seat_Simplified",
    "400_HIII",
    "500_NoCAB",
    "600_NoDoor",
    "700_Simplified_Belt",
    "800_Simplified_BIW",
    "900_NoIntrusion",
    "950_Dash_Rigid",
    "990_Carpet_Rigid",
]
REPORT_ASS = ["HW TH Report", "CAE TH Report"]
LVL_PACKAGES = (None, [0], [0, 1], *[[0, i, i + 1] for i in range(1, len(CASES) - 1)])
LOAD_CASES = ("Full Frontal", "Oblique Right", "Oblique Left", "Moderate Overlap Left", "Moderate Overlap Right")
T_MAX: float = 130

In [3]:
# read report data
csv = Csv(csv_path=B_PATH / "From_Reports" / "report_data.csv.zip", compress=True)
DB_REPORT: pd.DataFrame = csv.read()
DB_REPORT.replace({"CAE NHTSA": REPORT_ASS[1], "Test NHTSA": REPORT_ASS[0]}, inplace=True)
DB_REPORT = DB_REPORT[DB_REPORT["Time"].le(T_MAX)].copy()
display(DB_REPORT)

2024-06-03 13:11:42,588 MainProcess - MainThread INFO Read /mnt/q/Val_Chain_Sims/AB_Testing/From_Reports/report_data.csv.zip || Csv.py


,Value,Time,Channel,Source,Case
0,-0.2951,1.0,00COG00000VH00ACXD,CAE TH Report,Full Frontal
1,0.0105,2.0,00COG00000VH00ACXD,CAE TH Report,Full Frontal
2,0.0940,3.0,00COG00000VH00ACXD,CAE TH Report,Full Frontal
3,-0.1007,4.0,00COG00000VH00ACXD,CAE TH Report,Full Frontal
4,-1.3470,5.0,00COG00000VH00ACXD,CAE TH Report,Full Frontal
...,...,...,...,...,...
16944,0.4938,126.0,00COG00000VH00ACRD,HW TH Report,Oblique Right
16945,0.0572,127.0,00COG00000VH00ACRD,HW TH Report,Oblique Right
16946,0.1605,128.0,00COG00000VH00ACRD,HW TH Report,Oblique Right
16947,0.1341,129.0,00COG00000VH00ACRD,HW TH Report,Oblique Right


In [4]:
# read Other Data
DB_CAES: List[pd.DataFrame] = []
for case in CASES:
    LOG.info("Read %s", case)

    CSV = Csv(csv_path=B_PATH / case / f"extracted.csv.zip", compress=True)
    DB_CAES.append(CSV.read())

    DB_CAES[-1].replace(
        {"Assemblies_1": "SP 48", "Assemblies_2": "DP 48", "Assemblies_3": "SP 96", "Assemblies_4": "DP 96"}, inplace=True
    )

    DB_CAES[-1].fillna(0, inplace=True)

    DB_CAES[-1].replace({c: c.replace("_", " ") for c in DB_CAES[-1]["Case"]}, inplace=True)
    DB_CAES[-1] = DB_CAES[-1][DB_CAES[-1]["Time"].le(T_MAX)]

DB_CAES[-1]

2024-06-03 13:11:42,704 MainProcess - MainThread INFO Read 000_Base_Model || 1498813655.py
2024-06-03 13:11:42,705 MainProcess - MainThread INFO Read /mnt/q/Val_Chain_Sims/AB_Testing/000_Base_Model/extracted.csv.zip || Csv.py
2024-06-03 13:12:19,391 MainProcess - MainThread INFO Read 100_Guided_BIW || 1498813655.py
2024-06-03 13:12:19,393 MainProcess - MainThread INFO Read /mnt/q/Val_Chain_Sims/AB_Testing/100_Guided_BIW/extracted.csv.zip || Csv.py
2024-06-03 13:12:31,597 MainProcess - MainThread INFO Read 200_PAB_Simplified || 1498813655.py
2024-06-03 13:12:31,599 MainProcess - MainThread INFO Read /mnt/q/Val_Chain_Sims/AB_Testing/200_PAB_Simplified/extracted.csv.zip || Csv.py
2024-06-03 13:12:43,590 MainProcess - MainThread INFO Read 300_Seat_Simplified || 1498813655.py
2024-06-03 13:12:43,591 MainProcess - MainThread INFO Read /mnt/q/Val_Chain_Sims/AB_Testing/300_Seat_Simplified/extracted.csv.zip || Csv.py
2024-06-03 13:12:55,907 MainProcess - MainThread INFO Read 400_HIII || 1498813

,Time,Value,Channel,Source,Assembly,Configuration,Case,Side
0,0.0,-15.5594,00COG00000VH00VEXD,CAE THI,SP 48,990_Carpet_Rigid,Full Frontal,0.0
1,0.1,-15.5595,00COG00000VH00VEXD,CAE THI,SP 48,990_Carpet_Rigid,Full Frontal,0.0
2,0.2,-15.5596,00COG00000VH00VEXD,CAE THI,SP 48,990_Carpet_Rigid,Full Frontal,0.0
3,0.3,-15.5598,00COG00000VH00VEXD,CAE THI,SP 48,990_Carpet_Rigid,Full Frontal,0.0
4,0.4,-15.5599,00COG00000VH00VEXD,CAE THI,SP 48,990_Carpet_Rigid,Full Frontal,0.0
...,...,...,...,...,...,...,...,...
2975619,129.6,-64.6607,03PELVLOC0H350DSZD,CAE THI,DP 96,990_Carpet_Rigid,Moderate Overlap Right,0.0
2975620,129.7,-64.6930,03PELVLOC0H350DSZD,CAE THI,DP 96,990_Carpet_Rigid,Moderate Overlap Right,0.0
2975621,129.8,-64.7247,03PELVLOC0H350DSZD,CAE THI,DP 96,990_Carpet_Rigid,Moderate Overlap Right,0.0
2975622,129.9,-64.7559,03PELVLOC0H350DSZD,CAE THI,DP 96,990_Carpet_Rigid,Moderate Overlap Right,0.0


# Plot

In [5]:
def label(idx: int) -> str:
    return " ".join(CASES[idx].split("_")[1:])

In [6]:
def plot_pulse(case: Literal["Oblique Right", "Oblique Left"]):
    boundary_channels = [
        ["00COG00000VH00VEXD", "00COG00000VH00VEYD"],
        ["00COG00000VH00ACXD", "00COG00000VH00ACYD"],
    ]

    for lvl_package in LVL_PACKAGES:
        plot = ValChainStyledPlotCorridor(channels=boundary_channels, hw_ration=0.7, nb_path=P_PATH, t_max=T_MAX)

        for ch in plot.channels:
            # add report data (always CFC60)
            report = DB_REPORT[DB_REPORT["Channel"].eq(ch) & DB_REPORT["Case"].eq(case)]
            for i, ass in enumerate(REPORT_ASS, 1):
                report2 = report[report["Source"].eq(ass)]
                flip = (
                    -1
                    if (ch == "00COG00000VH00VEXD" and case != "Full Frontal")
                    or (ch == "00COG00000VH00ACXD")
                    or (ch == "00COG00000VH00VEYD" and case == "Oblique Left")
                    or (ch == "00COG00000VH00ACYD" and case == "Oblique Right")
                    else 1
                )
                if report2.shape[0]:
                    plot.add_line(channel=ch, x=report2["Time"], y=report2["Value"] * flip, label=ass, c_id=-i)

            # determine cae flip
            flip = (
                -1
                if (ch == "00COG00000VH00VEXD")
                or (ch == "00COG00000VH00ACXD")
                or (ch == "00COG00000VH00VEYD" and case == "Oblique Left")
                or (ch == "00COG00000VH00VEYD" and case == "Oblique Right")
                or (ch == "00COG00000VH00ACYD" and case == "Oblique Right")
                or (ch == "00COG00000VH00ACYD" and case == "Oblique Left")
                else 1
            )

            # filter CAE data
            isi = [] if lvl_package is None else lvl_package
            for c_i, i in enumerate(isi):
                db_cae = DB_CAES[i]
                flip2 = 1 if i == 0 else -1
                cae = db_cae[db_cae["Channel"].eq(ch) & db_cae["Case"].eq(case)]
                cae.loc[:, "Value"] *= flip * flip2

                if cae.shape[0]:
                    # plot single curves
                    for j, setting in enumerate(sorted(cae["Assembly"].unique())):
                        cae2 = cae[cae["Assembly"].eq(setting)].sort_values("Time")
                        plot.add_line(
                            channel=ch,
                            x=cae2["Time"],
                            y=cae2["Value"],
                            c_id=c_i,
                            ls_id=j,
                            label="",
                        )

                    # min max corridor
                    plot.add_range(channel=ch, cae=cae, fill_c_id=c_i, label=label(i))

        vs_name = "_vs_".join([str(x) for x in isi])
        plot.format_store(region="VEH_" + vs_name, case=case)
        plt.close(plot.fig)

In [7]:
for case in LOAD_CASES:
    plot_pulse(case)

2024-06-03 13:14:16,890 MainProcess - MainThread INFO Store plots in /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors with file extention(s) ['png', 'pdf'] || ValChainStyledPlotCorridor.py
2024-06-03 13:14:17,031 MainProcess - MainThread INFO Store /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors/PNG/FF__VEH_.png || ValChainStyledPlotCorridor.py
2024-06-03 13:14:17,689 MainProcess - MainThread INFO Store /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors/PDF/FF__VEH_.pdf || ValChainStyledPlotCorridor.py
2024-06-03 13:14:18,572 MainProcess - MainThread INFO Store plots in /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors with file extention(s) ['png', 'pdf'] || ValChainStyledPlotCorridor.py
2024-06-03 13:14:27,558 MainProcess - MainThread INFO Store /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors/PNG/FF__VEH_0.png || ValChainStyledPlotCorridor.py
2024-06-03 13:

In [8]:
def plt_restraint(side: Literal[1, 3], case: Literal["Oblique Right", "Oblique Left"]):
    s = {1: "01", 3: "03"}[side]

    channels = [
        [f"{s}FAB00000VH00PRRD", f"{s}BELTBUSLVH00DSRD"],
        [f"{s}BELTB000VH00DSRD", f"{s}BELTB000VH00FORD"],
        [f"{s}BELTB300VH00FORD", f"{s}BELTB400VH00FORD"],
        [f"{s}BELTB500VH00FORD", f"{s}BELTB600VH00FORD"],
    ]
    for lvl_package in LVL_PACKAGES:
        plot = ValChainStyledPlotCorridor(channels=channels, hw_ration=1.5, nb_path=P_PATH, t_max=T_MAX)
        for ch in plot.channels:
            # plot report data
            chh = f"{ch[:10]}TH50{ch[14:-1]}D"
            report = DB_REPORT[DB_REPORT["Channel"].eq(chh) & DB_REPORT["Case"].eq(case)]
            for i, ass in enumerate(REPORT_ASS, 1):
                report2 = report[report["Source"].eq(ass)]
                if report2.shape[0]:
                    plot.add_line(channel=ch, x=report2["Time"], y=report2["Value"], label=ass, c_id=-i)

            # plot CAE data
            isi = [] if lvl_package is None else lvl_package
            for c_i, i in enumerate(isi):
                db_cae = DB_CAES[i]
                # filter CAE data
                cae = db_cae[db_cae["Channel"].eq(ch) & db_cae["Case"].eq(case)]
                if "BELTB0" in ch and "DS" in ch:
                    cae["Value"] -= cae["Value"] .min()

                if cae.shape[0]:
                    # plot single curves
                    for j, setting in enumerate(sorted(cae["Assembly"].unique())):
                        cae2 = cae[cae["Assembly"].eq(setting)].sort_values("Time")
                        plot.add_line(
                            channel=ch, x=cae2["Time"], y=cae2["Value"], label="", c_id=c_i, ls_id=j
                        )

                    # min max corridor
                    plot.add_range(channel=ch, cae=cae, fill_c_id=c_i, label=label(idx=i))

        vs_name = "_vs_".join([str(x) for x in isi])
        plot.format_store(region="RESTRAINT_" + vs_name, case=case, side=side)
        plt.close(plot.fig)

In [9]:
for ss in [3]:
    for case in LOAD_CASES:
        plt_restraint(ss, case)

2024-06-03 13:30:29,122 MainProcess - MainThread INFO Store plots in /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors with file extention(s) ['png', 'pdf'] || ValChainStyledPlotCorridor.py
2024-06-03 13:30:29,360 MainProcess - MainThread INFO Store /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors/PNG/FF_PA_RESTRAINT_.png || ValChainStyledPlotCorridor.py
2024-06-03 13:30:30,608 MainProcess - MainThread INFO Store /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors/PDF/FF_PA_RESTRAINT_.pdf || ValChainStyledPlotCorridor.py
2024-06-03 13:30:32,038 MainProcess - MainThread INFO Store plots in /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors with file extention(s) ['png', 'pdf'] || ValChainStyledPlotCorridor.py
/tmp/ipykernel_705647/724926203.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [10]:
def plot_body_acc(side: Literal[1, 3], case: Literal["Oblique Right", "Oblique Left"], part: Literal["HEAD", "CHST", "PELV"]):
    s = {1: "01", 3: "03"}[side]

    channels = [
        [f"{s}{part}0000??50ACRD", f"{s}{part}0000??50ACXD"],
        [f"{s}{part}0000??50ACYD", f"{s}{part}0000??50ACZD"],
    ]
    for lvl_package in LVL_PACKAGES:
        plot = ValChainStyledPlotCorridor(channels=channels, hw_ration=0.8, nb_path=P_PATH, t_max=T_MAX)

        for ch in plot.channels:
            # plot report data
            report = DB_REPORT[DB_REPORT["Channel"].eq(ch.replace("??", "TH")) & DB_REPORT["Case"].eq(case)]
            flip = -1 if "HEAD0000??50ACX" in ch or "HEAD0000??50ACZ" in ch else 1

            for i, ass in enumerate(REPORT_ASS, 1):
                report2 = report[report["Source"].eq(ass)]
                if report2.shape[0]:
                    plot.add_line(channel=ch, x=report2["Time"], y=report2["Value"] * flip, label=ass, c_id=-i)

            # plot CAE data
            isi = [] if lvl_package is None else lvl_package
            for c_i, i in enumerate(isi):
                db_cae = DB_CAES[i]
                # filter CAE data
                cae = db_cae[db_cae["Channel"].eq(ch.replace("??", "TH" if i < 4 else "H3")) & db_cae["Case"].eq(case)]

                if cae.shape[0]:
                    # plot single curves
                    for j, setting in enumerate(sorted(cae["Assembly"].unique())):
                        cae2 = cae[cae["Assembly"].eq(setting)].sort_values("Time")
                        plot.add_line(channel=ch, x=cae2["Time"], y=cae2["Value"], label="", c_id=c_i, ls_id=j)

                    # min max corridor
                    plot.add_range(channel=ch, cae=cae, fill_c_id=c_i, label=label(idx=i))

        vs_name = "_vs_".join([str(x) for x in isi])
        plot.format_store(region=f"{part}_acceleration" + vs_name, case=case, side=side)
        plt.close(plot.fig)

In [11]:
for ss in [3]:
    for case in LOAD_CASES:
        for part in ["HEAD", "CHST", "PELV"]:
            plot_body_acc(ss, case, part)

2024-06-03 14:02:44,267 MainProcess - MainThread INFO Store plots in /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors with file extention(s) ['png', 'pdf'] || ValChainStyledPlotCorridor.py
2024-06-03 14:02:44,429 MainProcess - MainThread INFO Store /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors/PNG/FF_PA_HEAD_ACCELERATION.png || ValChainStyledPlotCorridor.py
2024-06-03 14:02:45,118 MainProcess - MainThread INFO Store /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors/PDF/FF_PA_HEAD_ACCELERATION.pdf || ValChainStyledPlotCorridor.py
2024-06-03 14:02:45,802 MainProcess - MainThread INFO Store plots in /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors with file extention(s) ['png', 'pdf'] || ValChainStyledPlotCorridor.py
2024-06-03 14:02:54,726 MainProcess - MainThread INFO Store /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors/PNG/FF_PA_HEAD_ACCELERATION0.png ||

In [12]:
def plot_femur_fo(side: Literal[1, 3], case: Literal["Oblique Right", "Oblique Left"]):
    s = {1: "01", 3: "03"}[side]

    channels = [
        [f"{s}FEMRLE00??50FORD", f"{s}FEMRRI00??50FORD"],
    ]
    for lvl_package in LVL_PACKAGES:
        plot = ValChainStyledPlotCorridor(channels=channels, hw_ration=0.5, nb_path=P_PATH, t_max=T_MAX)
        for ch in plot.channels:
            # plot report data
            report = DB_REPORT[DB_REPORT["Channel"].eq(ch.replace("??", "TH")) & DB_REPORT["Case"].eq(case)]
            for i, ass in enumerate(REPORT_ASS, 1):
                report2 = report[report["Source"].eq(ass)]
                if report2.shape[0]:
                    plot.add_line(channel=ch, x=report2["Time"], y=report2["Value"], label=ass, c_id=-i)

            # plot CAE data
            isi = [] if lvl_package is None else lvl_package
            for c_i, i in enumerate(isi):
                db_cae = DB_CAES[i]
                # filter CAE data
                cae = db_cae[db_cae["Channel"].eq(ch.replace("??", "TH" if i < 4 else "H3")) & db_cae["Case"].eq(case)]
                cae.loc[:, "Value"] *= 1 if i < 4 else -1

                if cae.shape[0]:
                    # plot single curves
                    for j, setting in enumerate(sorted(cae["Assembly"].unique())):
                        cae2 = cae[cae["Assembly"].eq(setting)].sort_values("Time")
                        plot.add_line(channel=ch, x=cae2["Time"], y=cae2["Value"], label="", c_id=c_i, ls_id=j)

                    # min max corridor
                    plot.add_range(channel=ch, cae=cae, fill_c_id=c_i, label=label(i))

        vs_name = "_vs_".join([str(x) for x in isi])
        plot.format_store(region="FEMR_force" + vs_name, case=case, side=side)
        plt.close(plot.fig)

In [13]:
for ss in [3]:
    for case in LOAD_CASES:
        plot_femur_fo(ss, case)

2024-06-03 14:51:15,636 MainProcess - MainThread INFO Store plots in /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors with file extention(s) ['png', 'pdf'] || ValChainStyledPlotCorridor.py
2024-06-03 14:51:15,756 MainProcess - MainThread INFO Store /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors/PNG/FF_PA_FEMR_FORCE.png || ValChainStyledPlotCorridor.py
2024-06-03 14:51:16,100 MainProcess - MainThread INFO Store /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors/PDF/FF_PA_FEMR_FORCE.pdf || ValChainStyledPlotCorridor.py
2024-06-03 14:51:16,488 MainProcess - MainThread INFO Store plots in /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors with file extention(s) ['png', 'pdf'] || ValChainStyledPlotCorridor.py
2024-06-03 14:51:21,102 MainProcess - MainThread INFO Store /mnt/q/Val_Chain_Sims/AB_Testing/corridors/054_Eval_ValChain_All_Corridors/PNG/FF_PA_FEMR_FORCE0.png || ValChainStyledPlotCo